In [2]:
import numpy as np
import torch
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F
import os

In [3]:

import sys
import numpy as np
# add gensim source code fro this path
sys.path.insert(0, "../../../")
sys.path.insert(0, "../../gensim/")

# import modules & set up logging
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
                
                

## Import the training data from file

In [4]:
from smart_open import smart_open
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in smart_open(os.path.join(self.dirname, fname), 'r'):
                yield line.split()

In [7]:
# point to sentences to be learned file here
my_sentences_fn = '../..//ULL/data/Practical2/wa/english_full/'
my_sentences_fn = '../..//ULL/data/Practical2/wa/english/'

sentences = MySentences(my_sentences_fn) # a memory-friendly iterator
print(list(sentences))

[['each', 'of', 'them', 'is', 'very', 'complex', ',', 'but', 'the', 'link', 'between', 'the', 'two', 'is', 'even', 'more', 'complex', 'which', 'makes', 'the', 'whole', 'situation', 'for', 'most', 'people', 'understandably', 'confusing', '.'], ['the', 'commissionners', 'went', 'on', ':'], ['they', 'are', 'constrained', 'by', 'limits', 'which', 'are', 'imposed', 'in', 'order', 'to', 'ensure', 'that', 'the', 'freedom', 'of', 'one', 'person', 'does', 'not', 'violate', 'that', 'of', 'another', '.'], ['it', 'made', 'the', 'buying', 'and', 'selling', 'of', 'sex', 'on', 'our', 'streets', 'legal', 'for', 'all', 'intents', 'and', 'purposes', '.'], ['the', 'way', 'in', 'which', 'Bill', 'C', '-', '49', 'has', 'been', 'presented', 'today', 'gives', 'us', 'the', 'feeling', 'that', 'only', 'the', 'prostitutes', 'are', 'punished', 'and', 'stigmatized', '.'], ['I', 'never', 'met', 'a', 'street', 'hooker', 'who', 'wanted', 'to', 'be', 'there', '.'], ['the', 'establishment', 'of', 'the', 'standing', 'com

In [8]:

# corpus = [
#     'he is a king',
#     'she is a queen',
#     'he is a man',
#     'she is a woman',
#     'warsaw is poland capital',
#     'berlin is germany capital',
#     'paris is france capital',
# ]


# def tokenize_corpus(corpus):
#     tokens = [x.split() for x in corpus]
#     return tokens

# tokenized_corpus = tokenize_corpus(corpus)


vocabulary = []
for sentence in sentences:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}


vocabulary_size = len(vocabulary)

In [9]:
for i in vocabulary:
    if i == 'he':
        print(i)

In [10]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in sentences:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array



In [11]:

def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [12]:
print(np.shape(W2))

NameError: name 'W2' is not defined

In [13]:
import time
start_time = time.time()

end_time=time.time()
print("Took %.2f"%(end_time-start_time))

Took 0.00


In [14]:
import time
start_time = time.time()

embedding_dims = 5
# D: Latent variables / embed dimensions
# V: vocabulary size
# W1: shape(VxD)
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
num_epochs = 101
learning_rate = 0.001


for epo in range(num_epochs):
    loss_val = 0
    # Compute loss for each training pair after predicting it with the NN
    for data, target in idx_pairs:
        # input layer: hot vectors?
        x = Variable(get_input_layer(data)).float()
        #print(data,target,  x)
        y_true = Variable(torch.from_numpy(np.array([target])).long())
        
        # this selects the embedding corresponding to x hot vector
        # z1: size(1xZ) column vec
        z1 = torch.matmul(W1, x)
        
        # multiply selected embedding, z1 with contexts matrix
        # computes the actual response z2 = yhat
        z2 = torch.matmul(W2, z1)
        
        # pass z2 through softmax to get probabilities
        # and compute log
        log_softmax = F.log_softmax(z2, dim=0)
        
        # ERROR: subtract model predictions from true contexts
        # print(log_softmax.view(1,-1))
        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        #print("loss",loss.data[0])
        loss_val += loss.data[0]
        # Run backprop - backprogate error 
        loss.backward()
        # Compute gradient of the parameters matrix W1, evaluate at data
        
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
    if epo % 10 == 0:    
        print('Loss at epo %s: %s/%s'%(epo, loss_val, len(idx_pairs)))
        

end_time=time.time()
print("Took %.2f"%(end_time-start_time))

/Users/efiathieniti/miniconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:39: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Loss at epo 0: tensor(19535.4629)/2422
Loss at epo 10: tensor(17245.0137)/2422
Loss at epo 20: tensor(16259.0898)/2422
Loss at epo 30: tensor(15661.2754)/2422
Loss at epo 40: tensor(15238.1240)/2422
Loss at epo 50: tensor(14910.4189)/2422
Loss at epo 60: tensor(14641.3564)/2422
Loss at epo 70: tensor(14411.1230)/2422
Loss at epo 80: tensor(14208.2285)/2422
Loss at epo 90: tensor(14025.7725)/2422
Loss at epo 100: tensor(13859.3662)/2422
Took 46.61


In [15]:
print(vocabulary_size)
# Embeddings are the W1

print(vocabulary[1])

322
of


In [16]:
W1.detach().numpy()[:,1]

array([ 0.1363363 ,  0.39781931,  0.25489193, -0.38880429,  0.54477477], dtype=float32)

## Save the embeddings learned as gensim model

In [17]:
import csv
embeddings_file='embeddings_simple.txt'
with open(embeddings_file, 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter = ' ')
    writer.writerow([vocabulary_size, embedding_dims])
    for i in range(vocabulary_size):    
        line= [vocabulary[i]] + list(W1.detach().numpy()[:,i])
        writer.writerow(line)
    

In [48]:
embeddings_file='embeddings_simple.txt'
embeddings_file='embeddings_new.txt'


## Load the embeddings learned 

In [49]:

from gensim.models import KeyedVectors
#word_vectors = KeyedVectors.load_word2vec_format('../../../ULL/data/bow2.words.bz2', binary=False)  # C text format
word_vectors = KeyedVectors.load_word2vec_format(embeddings_file, binary=False)  # C binary format

2018-05-02 16:39:33,402 : INFO : loading projection weights from embeddings_new.txt
2018-05-02 16:39:33,429 : INFO : loaded (1732, 5) matrix from embeddings_new.txt


In [20]:
for i in vocabulary:
    print(i)

each
of
them
is
very
complex
,
but
the
link
between
two
even
more
which
makes
whole
situation
for
most
people
understandably
confusing
.
commissionners
went
on
:
they
are
constrained
by
limits
imposed
in
order
to
ensure
that
freedom
one
person
does
not
violate
another
it
made
buying
and
selling
sex
our
streets
legal
all
intents
purposes
way
Bill
C
-
49
has
been
presented
today
gives
us
feeling
only
prostitutes
punished
stigmatized
I
never
met
a
street
hooker
who
wanted
be
there
establishment
standing
committee
unanimous
consent
House
June
28
1985
already
noted
have
stressed
CDC
as
an
example
creative
aggressive
effective
public
ownership
so
far
around
$
250,000
issued
institute
grants
research
or
education
activity
let
me
conclude
think
we
should
take
hard
look
at
this
justification
determine
whether
cost
interest
however
raises
some
pertinent
questions
given
cut
backs
announced
Budget
last
spring
see
licence
fee
going
up
from
25
500
would
easy
say
these
sanctions
followed
rather
than


In [50]:
print(word_vectors.similarity('disturbed', 'pervasive'))


0.64288


In [51]:
word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
[('queen', 0.50882536), ...]

word_vectors.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
[('queen', 0.71382287), ...]

word_vectors.doesnt_match("breakfast cereal dinner lunch".split())
'cereal'



2018-05-02 16:39:46,292 : INFO : precomputing L2-norms of word weight vectors


KeyError: "word 'woman' not in vocabulary"

In [52]:
word_vectors.doesnt_match("breakfast cereal dinner lunch".split())
'cereal'


2018-05-02 16:39:48,912 : WARNING : vectors for words {'dinner', 'cereal', 'lunch', 'breakfast'} are not present in the model, ignoring these words


ValueError: cannot select a word from an empty list

In [53]:
word_vectors.similarity('woman', 'man')


KeyError: "word 'woman' not in vocabulary"

Evaluate model, similarity, analogy

# Bayesian skip-gram

1. 
For each word/training point (xi) calculate elbo (loss)
xi - training points


In [46]:


# elbo: 
# calculate mu_xi and sigma_xi (length: V)
# log(z | mu_xi, sigma_xi)


import time
start_time = time.time()

embedding_dims = 5
# D: Latent variables / embed dimensions
# V: vocabulary size
# W1: shape(VxD)
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
num_epochs = 5
learning_rate = 0.001


for epo in range(num_epochs):
    print("epo ",epo)
    loss_val = 0
    # Compute loss for each training pair after predicting it with the NN
    for data, target in idx_pairs:
        print("data",data)
        # input layer: hot vectors?
        x_d = Variable(get_input_layer(data)).float()
        x_t = Variable(get_input_layer(target)).float()

        # this selects the embedding corresponding to x hot vector
        # z1: size(1xZ) column vec
        z_d = torch.matmul(W1, x_d)
        z_t = torch.matmul(W1, x_t)
        z = torch.cat((z_d, z_t), 0)
        #print(z)
        # Encoder
        z = F.relu(z)
        
        # During inference, we simply spit out the mean of the

        std = logvar.mul(0.5).exp_()  # type: Variable
        mu, logvar = self.encode(x.view(-1, 784))
        eps = Variable(std.data.new(std.size()).normal_())

        
        
        log_softmax = F.log_softmax(z2, dim=0)

        

epo  0
data 0
tensor([ 0.4625,  0.3210,  0.0000,  0.0000,  0.4706,  1.4971,  0.0000,
         0.7202,  0.0000,  0.6288])
data 0
tensor([ 0.4625,  0.3210,  0.0000,  0.0000,  0.4706,  0.0000,  0.6466,
         0.0000,  0.0000,  0.0701])
data 1
tensor([ 1.4971,  0.0000,  0.7202,  0.0000,  0.6288,  0.4625,  0.3210,
         0.0000,  0.0000,  0.4706])
data 1
tensor([ 1.4971,  0.0000,  0.7202,  0.0000,  0.6288,  0.0000,  0.6466,
         0.0000,  0.0000,  0.0701])
data 1
tensor([ 1.4971,  0.0000,  0.7202,  0.0000,  0.6288,  0.0000,  0.7732,
         1.4456,  0.0000,  0.5693])
data 2
tensor([ 0.0000,  0.6466,  0.0000,  0.0000,  0.0701,  0.4625,  0.3210,
         0.0000,  0.0000,  0.4706])
data 2
tensor([ 0.0000,  0.6466,  0.0000,  0.0000,  0.0701,  1.4971,  0.0000,
         0.7202,  0.0000,  0.6288])
data 2
tensor([ 0.0000,  0.6466,  0.0000,  0.0000,  0.0701,  0.0000,  0.7732,
         1.4456,  0.0000,  0.5693])
data 2
tensor([ 0.0000,  0.6466,  0.0000,  0.0000,  0.0701,  0.9038,  0.0000,
   

tensor([ 0.4853,  0.1077,  0.0766,  0.6611,  0.0000,  0.5246,  0.0000,
         0.0000,  0.0000,  0.8124])
data 64
tensor([ 0.4853,  0.1077,  0.0766,  0.6611,  0.0000,  0.3498,  0.0000,
         0.0000,  0.5190,  0.0539])
data 64
tensor([ 0.4853,  0.1077,  0.0766,  0.6611,  0.0000,  0.6653,  0.0862,
         0.3067,  0.0000,  0.0000])
data 64
tensor([ 0.4853,  0.1077,  0.0766,  0.6611,  0.0000,  0.0080,  0.0000,
         0.2101,  0.0000,  0.0000])
data 65
tensor([ 0.6653,  0.0862,  0.3067,  0.0000,  0.0000,  0.3498,  0.0000,
         0.0000,  0.5190,  0.0539])
data 65
tensor([ 0.6653,  0.0862,  0.3067,  0.0000,  0.0000,  0.4853,  0.1077,
         0.0766,  0.6611,  0.0000])
data 65
tensor([ 0.6653,  0.0862,  0.3067,  0.0000,  0.0000,  0.0080,  0.0000,
         0.2101,  0.0000,  0.0000])
data 65
tensor([ 0.6653,  0.0862,  0.3067,  0.0000,  0.0000,  0.0494,  0.1900,
         0.0000,  0.0000,  0.0000])
data 66
tensor([ 0.0080,  0.0000,  0.2101,  0.0000,  0.0000,  0.4853,  0.1077,
         

tensor([ 0.0000,  0.1369,  0.8353,  1.5460,  0.0000,  0.0000,  0.0000,
         1.7145,  0.0000,  0.0000])
data 8
tensor([ 0.0000,  0.1369,  0.8353,  1.5460,  0.0000,  0.6260,  0.5747,
         1.6286,  0.0000,  2.7496])
data 8
tensor([ 0.0000,  0.1369,  0.8353,  1.5460,  0.0000,  0.8327,  0.0000,
         0.0397,  0.5506,  1.0354])
data 112
tensor([ 0.6260,  0.5747,  1.6286,  0.0000,  2.7496,  0.0000,  0.0000,
         1.7145,  0.0000,  0.0000])
data 112
tensor([ 0.6260,  0.5747,  1.6286,  0.0000,  2.7496,  0.0000,  0.1369,
         0.8353,  1.5460,  0.0000])
data 112
tensor([ 0.6260,  0.5747,  1.6286,  0.0000,  2.7496,  0.8327,  0.0000,
         0.0397,  0.5506,  1.0354])
data 112
tensor([ 0.6260,  0.5747,  1.6286,  0.0000,  2.7496,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0200])
data 34
tensor([ 0.8327,  0.0000,  0.0397,  0.5506,  1.0354,  0.0000,  0.1369,
         0.8353,  1.5460,  0.0000])
data 34
tensor([ 0.8327,  0.0000,  0.0397,  0.5506,  1.0354,  0.6260,  0.5747,
       

tensor([ 0.6217,  0.9256,  0.4274,  0.8396,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000])
data 159
tensor([ 0.6217,  0.9256,  0.4274,  0.8396,  0.0000,  0.2162,  0.0000,
         0.7949,  0.0000,  0.4726])
data 159
tensor([ 0.6217,  0.9256,  0.4274,  0.8396,  0.0000,  0.2832,  0.0000,
         0.7465,  1.1789,  0.3843])
data 160
tensor([ 0.2162,  0.0000,  0.7949,  0.0000,  0.4726,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000])
data 160
tensor([ 0.2162,  0.0000,  0.7949,  0.0000,  0.4726,  0.6217,  0.9256,
         0.4274,  0.8396,  0.0000])
data 160
tensor([ 0.2162,  0.0000,  0.7949,  0.0000,  0.4726,  0.2832,  0.0000,
         0.7465,  1.1789,  0.3843])
data 160
tensor([ 0.2162,  0.0000,  0.7949,  0.0000,  0.4726,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000])
data 161
tensor([ 0.2832,  0.0000,  0.7465,  1.1789,  0.3843,  0.6217,  0.9256,
         0.4274,  0.8396,  0.0000])
data 161
tensor([ 0.2832,  0.0000,  0.7465,  1.1789,  0.3843,  0.2162,  0.0000,
 

tensor([ 0.2792,  1.3562,  0.0000,  0.2705,  0.0995,  0.3596,  2.2117,
         0.0000,  0.8276,  0.0000])
data 43
tensor([ 0.0000,  0.5715,  0.0000,  1.5224,  0.0000,  0.8176,  0.0000,
         0.4990,  0.5230,  0.0000])
data 43
tensor([ 0.0000,  0.5715,  0.0000,  1.5224,  0.0000,  0.2792,  1.3562,
         0.0000,  0.2705,  0.0995])
data 43
tensor([ 0.0000,  0.5715,  0.0000,  1.5224,  0.0000,  0.3596,  2.2117,
         0.0000,  0.8276,  0.0000])
data 43
tensor([ 0.0000,  0.5715,  0.0000,  1.5224,  0.0000,  0.1080,  0.8744,
         0.0000,  0.2145,  0.0000])
data 192
tensor([ 0.3596,  2.2117,  0.0000,  0.8276,  0.0000,  0.2792,  1.3562,
         0.0000,  0.2705,  0.0995])
data 192
tensor([ 0.3596,  2.2117,  0.0000,  0.8276,  0.0000,  0.0000,  0.5715,
         0.0000,  1.5224,  0.0000])
data 192
tensor([ 0.3596,  2.2117,  0.0000,  0.8276,  0.0000,  0.1080,  0.8744,
         0.0000,  0.2145,  0.0000])
data 192
tensor([ 0.3596,  2.2117,  0.0000,  0.8276,  0.0000,  0.0000,  0.2029,
     

...

tensor([ 1.5312,  0.8795,  2.1102,  1.9857,  0.0000,  0.3635,  0.0000,
         1.5981,  0.0000,  0.0000])
data 18
tensor([ 1.5312,  0.8795,  2.1102,  1.9857,  0.0000,  0.0000,  0.9860,
         0.0000,  0.7566,  0.8767])
data 167
tensor([ 0.3635,  0.0000,  1.5981,  0.0000,  0.0000,  0.0000,  1.3625,
         0.5639,  0.8215,  0.0000])
data 167
tensor([ 0.3635,  0.0000,  1.5981,  0.0000,  0.0000,  1.5312,  0.8795,
         2.1102,  1.9857,  0.0000])
data 167
tensor([ 0.3635,  0.0000,  1.5981,  0.0000,  0.0000,  0.0000,  0.9860,
         0.0000,  0.7566,  0.8767])
data 167
tensor([ 0.3635,  0.0000,  1.5981,  0.0000,  0.0000,  0.0000,  1.8276,
         0.0000,  0.6419,  0.0000])
data 168
tensor([ 0.0000,  0.9860,  0.0000,  0.7566,  0.8767,  1.5312,  0.8795,
         2.1102,  1.9857,  0.0000])
data 168
tensor([ 0.0000,  0.9860,  0.0000,  0.7566,  0.8767,  0.3635,  0.0000,
         1.5981,  0.0000,  0.0000])
data 168
tensor([ 0.0000,  0.9860,  0.0000,  0.7566,  0.8767,  0.0000,  1.8276,
  

tensor([ 0.0000,  0.0000,  0.0374,  0.1778,  1.3081,  0.6017,  0.5292,
         0.6238,  0.7343,  0.8866])
data 207
tensor([ 0.0000,  0.0000,  0.0374,  0.1778,  1.3081,  0.0000,  0.0000,
         2.7306,  0.0000,  0.0000])
data 23
tensor([ 0.0000,  0.0000,  2.7306,  0.0000,  0.0000,  0.6017,  0.5292,
         0.6238,  0.7343,  0.8866])
data 23
tensor([ 0.0000,  0.0000,  2.7306,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0374,  0.1778,  1.3081])
data 208
tensor([ 0.0000,  1.0955,  0.0000,  0.0893,  0.0000,  0.0000,  0.0000,
         0.0000,  0.4723,  0.0000])
data 208
tensor([ 0.0000,  1.0955,  0.0000,  0.0893,  0.0000,  0.3148,  0.0000,
         1.6240,  0.6385,  1.1458])
data 46
tensor([ 0.0000,  0.0000,  0.0000,  0.4723,  0.0000,  0.0000,  1.0955,
         0.0000,  0.0893,  0.0000])
data 46
tensor([ 0.0000,  0.0000,  0.0000,  0.4723,  0.0000,  0.3148,  0.0000,
         1.6240,  0.6385,  1.1458])
data 46
tensor([ 0.0000,  0.0000,  0.0000,  0.4723,  0.0000,  0.0000,  0.5715,
      

data 231
tensor([ 1.3172,  0.0000,  0.2850,  1.4147,  0.0000,  0.0000,  0.0000,
         1.1793,  0.0000,  0.0000])
data 231
tensor([ 1.3172,  0.0000,  0.2850,  1.4147,  0.0000,  0.0000,  0.5477,
         0.0000,  1.0770,  0.0000])
data 231
tensor([ 1.3172,  0.0000,  0.2850,  1.4147,  0.0000,  0.1080,  0.8744,
         0.0000,  0.2145,  0.0000])
data 231
tensor([ 1.3172,  0.0000,  0.2850,  1.4147,  0.0000,  0.3162,  1.3044,
         0.0000,  1.4172,  0.0000])
data 36
tensor([ 0.1080,  0.8744,  0.0000,  0.2145,  0.0000,  0.0000,  0.5477,
         0.0000,  1.0770,  0.0000])
data 36
tensor([ 0.1080,  0.8744,  0.0000,  0.2145,  0.0000,  1.3172,  0.0000,
         0.2850,  1.4147,  0.0000])
data 36
tensor([ 0.1080,  0.8744,  0.0000,  0.2145,  0.0000,  0.3162,  1.3044,
         0.0000,  1.4172,  0.0000])
data 36
tensor([ 0.1080,  0.8744,  0.0000,  0.2145,  0.0000,  0.0000,  0.0000,
         1.1793,  0.0000,  0.0000])
data 232
tensor([ 0.3162,  1.3044,  0.0000,  1.4172,  0.0000,  1.3172,  0.00

tensor([ 0.4185,  0.2911,  1.2268,  0.0518,  0.0000,  0.8327,  0.0000,
         0.0397,  0.5506,  1.0354])
data 272
tensor([ 0.4185,  0.2911,  1.2268,  0.0518,  0.0000,  0.0000,  0.1369,
         0.8353,  1.5460,  0.0000])
data 272
tensor([ 0.4185,  0.2911,  1.2268,  0.0518,  0.0000,  0.0000,  0.0000,
         2.7306,  0.0000,  0.0000])
data 23
tensor([ 0.0000,  0.0000,  2.7306,  0.0000,  0.0000,  0.0000,  0.1369,
         0.8353,  1.5460,  0.0000])
data 23
tensor([ 0.0000,  0.0000,  2.7306,  0.0000,  0.0000,  0.4185,  0.2911,
         1.2268,  0.0518,  0.0000])
data 28
tensor([ 1.4322,  1.1425,  0.0000,  0.0000,  0.0000,  0.3596,  2.2117,
         0.0000,  0.8276,  0.0000])
data 28
tensor([ 1.4322,  1.1425,  0.0000,  0.0000,  0.0000,  0.1080,  0.8744,
         0.0000,  0.2145,  0.0000])
data 192
tensor([ 0.3596,  2.2117,  0.0000,  0.8276,  0.0000,  1.4322,  1.1425,
         0.0000,  0.0000,  0.0000])
data 192
tensor([ 0.3596,  2.2117,  0.0000,  0.8276,  0.0000,  0.1080,  0.8744,
     

data 301
tensor([ 0.3734,  2.0300,  0.0000,  0.0000,  0.0000,  0.2971,  0.6734,
         0.1401,  0.0000,  0.0000])
data 301
tensor([ 0.3734,  2.0300,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000])
data 301
tensor([ 0.3734,  2.0300,  0.0000,  0.0000,  0.0000,  0.8327,  0.0000,
         0.0397,  0.5506,  1.0354])
data 302
tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.2971,  0.6734,
         0.1401,  0.0000,  0.0000])
data 302
tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.3734,  2.0300,
         0.0000,  0.0000,  0.0000])
data 302
tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.8327,  0.0000,
         0.0397,  0.5506,  1.0354])
data 302
tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.1495,  0.1095,
         1.0434,  0.1118,  0.0753])
data 34
tensor([ 0.8327,  0.0000,  0.0397,  0.5506,  1.0354,  0.3734,  2.0300,
         0.0000,  0.0000,  0.0000])
data 34
tensor([ 0.8327,  0.0000,  0.0397,  0.5506,  1.0354,  0.0000,  0.

In [31]:
z_t.size()

torch.Size([5])

In [47]:
z(10)

TypeError: 'Tensor' object is not callable